<a href="https://colab.research.google.com/github/wanwanliang/Image_Processing_and_Deep_Learning/blob/master/code/CNNs_WSR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Prepare data

## install and load packages

In [ ]:
!pip install geopandas
!pip install rasterio

In [4]:
import rasterio as rio
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import sklearn
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import google.colab
from google.colab import drive
import glob

In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
os.chdir("/content/drive/My Drive/UMN_Research/Data/wsr/image_200_bb45")

In [7]:
from keras.layers import Input, Conv2D, Activation, BatchNormalization, GlobalAveragePooling2D, Dense, Dropout, AveragePooling2D, Flatten, ZeroPadding2D, MaxPooling2D, Add
from keras.activations import relu, softmax
from keras.models import Model
from keras import regularizers, initializers

In [8]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# Design Resnet

### Define identity block

In [9]:
def identity_block(X,f,filters, stage, block):

  # defining name basis
  conv_name_base = 'res' + str(stage) + block + "_branch"
  bn_name_base = 'bn' + str(stage) + block + "_branch"

  F1, F2, F3 = filters

  X_shortcut = X

  # first component of main path
  X = Conv2D(filters=F1, kernel_size=(1,1), strides=(1,1), padding='valid', name = conv_name_base + '2a', kernel_initializer=initializers.glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
  X = Activation('relu')(X)

  # second component of main path
  X = Conv2D(filters=F2, kernel_size=(f,f), strides=(1,1), padding='same', name=conv_name_base + '2b', kernel_initializer=initializers.glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name=bn_name_base + '2b')(X)
  X =  Activation('relu')(X)

  # third component of main path
  X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = initializers.glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

  # final step
  X = Add()([X, X_shortcut])
  X = Activation('relu')(X)

  return X

### Define convolutional block

In [10]:
def convolutional_block(X, f, filters, stage, block, s=2):

  # defining name basis
  conv_name_base = 'res' + str(stage) + block + "_branch"
  bn_name_base = 'bn' + str(stage) + block + "_branch"

  # Retrieve Filters
  F1, F2, F3 = filters

  X_shortcut = X

  # First component of main path 
  X = Conv2D(F1, (1,1), strides = (s,s), name= conv_name_base + "2a", kernel_initializer= initializers.glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=3, name=bn_name_base + '2a')(X)
  X = Activation('relu')(X)

  # Second component of main path 
  X = Conv2D(filters= F2, kernel_size=(f,f), strides=(1,1), padding='same', name=conv_name_base + '2b', kernel_initializer= initializers.glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=3, name=bn_name_base + '2b')(X)
  X = Activation('relu')(X)

  # Third component of main path 
  X = Conv2D(filters = F3, kernel_size=(1,1), strides=(1,1), padding='valid', name=conv_name_base + '2c', kernel_initializer=initializers.glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

  ##### SHORTCUT PATH #### 
  X_shortcut =  Conv2D(F3, (1, 1), strides = (s,s), name = conv_name_base + '1', kernel_initializer = initializers.glorot_uniform(seed=0))(X_shortcut)
  X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

  # Final step: Add shortcut value to main path, and pass it through a RELU activation 
  X = Add()([X, X_shortcut])
  X = Activation('relu')(X)

  return X

### ResNet50

In [11]:
def ResNet50(input_shape = (64, 64, 5), classes=2):

  X_input = Input(input_shape)

  X = ZeroPadding2D((3,3))(X_input)

  # stage 1 
  X = Conv2D(64, (3,3), strides=(2,2), name='conv1', kernel_initializer=initializers.glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=3, name='bn_conv1')(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((3,3), strides=(2,2), padding='same')(X)
  
  # stage 2
  X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
  X = identity_block(X, 3, [64,64,256], stage=2, block='b')
  X = identity_block(X, 3, [64,64,256], stage=2, block='c')

  # stage 3
  X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
  X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
  X = identity_block(X, 3, [128,128,512], stage=3, block='c')
  X = identity_block(X, 3, [128,128,512], stage=3, block='d')

  # stage 4
  X = convolutional_block(X, f=3, filters=[256,256,1024], stage=4, block='a', s=2)
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

  # stage 5
  X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
  X = identity_block(X, 3, [512,512, 2048], stage=5, block='b')
  X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

  # averge pooling
  X = AveragePooling2D(name='avg_pool', padding='same')(X)

  # output layer
  X = Flatten()(X)
  X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer= initializers.glorot_uniform(seed=0))(X)

  # create model
  model = Model(inputs = X_input, outputs= X, name='ResNet50')

  return model

### ResNet18

In [12]:
def ResNet18(input_shape = (64, 64, 5), classes=2):

  X_input = Input(input_shape)

  X = ZeroPadding2D((3,3))(X_input)

  # stage 1 
  X = Conv2D(64, (3,3), strides=(2,2), name='conv1', kernel_initializer=initializers.glorot_uniform(seed=0))(X)
  X = BatchNormalization(axis=3, name='bn_conv1')(X)
  X = Activation('relu')(X)
  X = MaxPooling2D((3,3), strides=(1,1), padding='same')(X)
  
  # stage 2
  X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
  X = identity_block(X, 3, [64,64,256], stage=2, block='b')


  # stage 3
  X = convolutional_block(X, f=3, filters=[128, 128, 512], stage=3, block='a', s=2)
  X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')


  # stage 4
  X = convolutional_block(X, f=3, filters=[256,256,1024], stage=4, block='a', s=2)
  X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')


  # stage 5
  X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
  X = identity_block(X, 3, [512,512, 2048], stage=5, block='b')


  # averge pooling
  X = AveragePooling2D(name='avg_pool', padding='same')(X)

  # output layer
  X = Flatten()(X)
  X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer= initializers.glorot_uniform(seed=0))(X)

  # create model
  model = Model(inputs = X_input, outputs= X, name='ResNet50')

  return model

# Load and prepare data

In [13]:
os.chdir('/content/drive/Shared drives/WSR_data/Drone200ft/Multispectral_LargerPlotSize')

### list and order files

In [14]:
import glob
t = glob.glob("*.tif")
# t2 = sorted(t)
# os.remove('plot621 (1).tif')
print(len(t))
t[:10]

960


['plot598.tif',
 'plot482.tif',
 'plot624.tif',
 'plot574.tif',
 'plot520.tif',
 'plot560.tif',
 'plot636.tif',
 'plot501.tif',
 'plot541.tif',
 'plot587.tif']

In [15]:
nbs = []
[nbs.append(int((td.split('plot')[1]).split('.')[0])) for td in t]
nbs[:10]

[598, 482, 624, 574, 520, 560, 636, 501, 541, 587]

In [16]:
all = zip(nbs, t)
sorted_all = sorted(all)

In [17]:
t_sorted = [x for y, x in sorted_all]

In [18]:
t_sorted[:10]

['plot1.tif',
 'plot2.tif',
 'plot3.tif',
 'plot4.tif',
 'plot5.tif',
 'plot6.tif',
 'plot7.tif',
 'plot8.tif',
 'plot9.tif',
 'plot10.tif']

### load all images in one numpy array

In [19]:
def tif2ary(tif):

  raA = rio.open(tif)
  arys = raA.read()

  arys= arys.astype('float32')
  arys =np.moveaxis(arys, 0, -1)

  
  return(arys)

In [20]:
dt = None

for tif in t_sorted:

  ary = tif2ary(tif)
  ary = ary[1:33, 1:33:,]

  # resize image array to 64*64
  ary = np.repeat(ary, 2, axis=1)
  ary = np.repeat(ary, 2, axis=0)

  ary = ary.reshape((1, 64, 64,5))

  if dt is None:
    dt = ary
  else:
    dt = np.concatenate((dt, ary), axis=0)

In [21]:
dt.shape

(960, 64, 64, 5)

In [22]:
min_val = [np.min(dt[:,:,:,i]) for i in range(5)]
print(min_val)

[3707.8135, 3668.7615, 2762.414, 12326.874, 3777.8079]


In [23]:
max_val = [np.max(dt[:,:,:,i]) for i in range(5)]
print(max_val)
dt = dt/max_val
print([np.max(dt[:,:,:,i]) for i in range(5)])

[64268.277, 60118.1, 35737.86, 236070.98, 52668.152]
[1.0, 1.0, 1.0, 1.0, 1.0]


In [24]:
y = pd.read_csv("../labels.csv")
print(y.shape)
y[:5]

(960, 8)


,plot_ID,binary_1,score,resistance,resistance_class_4,binary_2,block,variety
0,1,1,50.0,S,S,S,2,DH058
1,2,1,5.0,RMR,MR,R,2,Faller
2,3,1,25.0,MSS,S,S,2,DH121
3,4,1,40.0,MSS,S,S,2,DH80
4,5,1,25.0,MSS,S,S,2,ROB


In [25]:
y.binary_1.value_counts()

0    483
1    477
Name: binary_1, dtype: int64

In [26]:
y = y['binary_1']
y = np.asarray(y).reshape((-1,1))
y.shape

(960, 1)

In [27]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
encoder.fit(y)
y = encoder.transform(y)
y.shape

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


(960,)

In [28]:
x_train, x_test, y_train, y_test = train_test_split(dt, y, random_state=16, shuffle=True, test_size=0.1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state=16, test_size=0.15)

print("Train size is: {}".format(x_train.shape[0]))
print("Test size is: {}".format(x_test.shape[0]))
print("Validation size is: {}".format(x_val.shape[0]))

Train size is: 734
Test size is: 96
Validation size is: 130


In [29]:
x_train.shape

(734, 64, 64, 5)

In [30]:
ts = x_train[0]
print(np.min(ts))
print(np.max(ts))

0.118229024
0.72910243



# DL Model 

## Model Training

In [42]:
model = ResNet50(input_shape=(64, 64, 5), classes=2)
#model = ResNet18(input_shape=(64, 64, 5), classes=2)

In [43]:
model.compile(optimizer= tf.keras.optimizers.Adam(lr=0.005),loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [44]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20),
    tf.keras.callbacks.ModelCheckpoint(filepath='./models/ResNet50/model.{epoch:02d}-{val_loss:.2f}.h5', save_best_only=False, monitor='val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, verbose=1, epsilon=1e-4, min_lr = 0.000001, mode='auto')
]

In [ ]:
model.fit(x_train, y_train, epochs=200, batch_size=24, validation_data=(x_val, y_val), callbacks=my_callbacks)

Epoch 1/200
31/31 [==============================] - 101s 3s/step - loss: 9.5238 - accuracy: 0.5240 - val_loss: 1519323136.0000 - val_accuracy: 0.4923
Epoch 2/200
31/31 [==============================] - 95s 3s/step - loss: 1.0657 - accuracy: 0.4637 - val_loss: 17.3618 - val_accuracy: 0.3923
Epoch 3/200
31/31 [==============================] - 95s 3s/step - loss: 1.1520 - accuracy: 0.5185 - val_loss: 767.5388 - val_accuracy: 0.5077
Epoch 4/200
31/31 [==============================] - 94s 3s/step - loss: 1.1175 - accuracy: 0.5137 - val_loss: 0.7417 - val_accuracy: 0.4923
Epoch 5/200
31/31 [==============================] - 95s 3s/step - loss: 0.9253 - accuracy: 0.5635 - val_loss: 29.2690 - val_accuracy: 0.4846
Epoch 6/200
31/31 [==============================] - 95s 3s/step - loss: 0.7416 - accuracy: 0.6219 - val_loss: 0.6788 - val_accuracy: 0.6538
Epoch 7/200
31/31 [==============================] - 94s 3s/step - loss: 0.7204 - accuracy: 0.6581 - val_loss: 0.7240 - val_accuracy: 0.4923

## Model Evaluation

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from keras_retinanet.models import load_model

In [ ]:
os.chdir('/content/drive/Shared drives/WSR_data/Drone200ft/Multispectral_LargerPlotSize/models/ResNet18')

best_model = './modelBest2.h5'
model = load_model(best_model)

In [ ]:
#y_test_pred = model.predict_on_batch(x_test)
model.evaluate(x_test)

In [ ]:
y_test_pred = model.predict(x_test)
print('Testing accuracy is 

{}'.format(accuracy_ssocre(y_test, y_test_pred)))

print ('Confusion matrix:')
print(confusion_matrix(y_test, y_test_pred))

# ML Models

In [ ]:
from sklearn.ensemble import BaggingClassifier as bc
from sklearn.tree import DecisionTreeClassifier as dtc
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.ensemble import VotingClassifier as vc
from sklearn.linear_model import LogisticRegression as lrc
from sklearn.svm import SVC as svc
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

## Read data

In [ ]:
os.chdir("/content/drive/My Drive/UMN_Research/Data/wsr")
dt = pd.read_csv('data.csv')
print(dt.shape)
dt.head(5)

## Data Clearning

In [ ]:
dt2=dt
dt2 = dt2.drop(['plotID'],axis=1)
dt2.shape

dt2['class2']=0
dt2['class2'].iloc[(dt2.block==2)|(dt2.block==3)|(dt2.block==6)]=1

dt2['class2'].value_counts()

In [ ]:
dt3 = dt2
dt3 = dt3.drop(['score','resistance_class_4','binary_2','block'],axis=1)
print(dt3.shape)
dt3.head(5)

### Check and remove NaN

In [ ]:
pd.isnull(dt3).any(axis=0)

In [ ]:
pd.isnull(dt3).sum(axis=0)

In [ ]:
dt4 = dt3.dropna(axis=0)
print(dt4.shape)

## Generate training, validation, and testing data

In [ ]:
x, y = dt4.drop(['class2'], axis=1), dt4['class2']
x = StandardScaler().fit_transform(x)
x.shape

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=16, shuffle=True, test_size=0.15)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, random_state=16, test_size=0.2)

print("Train size is: {}".format(x_train.shape[0]))
print("Test size is: {}".format(x_test.shape[0]))
print("Validation size is: {}".format(x_val.shape[0]))

## RF and SVM Models

In [ ]:
rf_clf = rfc(n_estimators=500, random_state=6) 
svm_clf = svc(gamma='scale', random_state=6)

rf_clf.fit(x_train, y_train)
svm_clf.fit(x_train, y_train)

In [ ]:
y_pred_rf = rf_clf.predict(x_test)
y_pred_svm = svm_clf.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score

print("Test Accuracy by RF: {}".format(accuracy_score(y_test, y_pred_rf)))
print("Test Accuracy by SVM: {}".format(accuracy_score(y_test, y_pred_svm)))

In [ ]:
from sklearn.metrics import confusion_matrix
print("Confusion matrix by RF:")
print(confusion_matrix(y_test, y_pred_rf))

print("")
print("Confusion matrix by SVM:")
print(confusion_matrix(y_test, y_pred_svm))

## MLP Model

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
mlp_model = keras.models.Sequential()
mlp_model.add(Dense(30, input_dim = 26, activation='relu'))
mlp_model.add(Dense(30, activation='relu'))
mlp_model.add(Dense(2, activation='sigmoid'))

In [ ]:
mlp_model.compile(loss='sparse_categorical_crossentropy',
                  optimizer = keras.optimizers.Adam(learning_rate=0.01),
                  metrics=['accuracy'])

In [ ]:
fit_history = mlp_model.fit(x_train, y_train, epochs=200,batch_size=32,verbose=0, validation_data=(x_val, y_val))

In [ ]:
pd.DataFrame(fit_history.history).plot(figsize=(10,6))
plt.grid(True)
plt.gca().set_ylim(0,1)
plt.show()

In [ ]:
y_pred_mlp = mlp_model.predict_classes(x_test)
print("Test Accuracy by MLP: {}".format(accuracy_score(y_test, y_pred_mlp)))

print("Confusion matrix by MLP:")
print(confusion_matrix(y_test, y_pred_mlp))